In [2]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load(disable=[ "ner","tagger",'textcat'])
from spellchecker import SpellChecker
import nltk
from nltk.tokenize import word_tokenize
import re
import string
import unicodedata
import emoji as emoji
import regex
import spacy
from spacy.tokens import Token
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from pandas import Panel
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
import time
import numpy as np
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score


C:\Users\asus\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version


## PREPROCESSING

In [3]:
##1
def remove_stopwords(text):
    text=text.lower()
    tokens=[token for token in word_tokenize(text) if token not in stopwords.words('english')]
    return ' '.join([w for w in tokens])
##2
def remove_Urls(text):
    return re.sub(r"http\S+", "", text)
## 3
def clear_spaces(text):
    text=text.replace("\t"," ")
    text = text.replace("\n", " ")
    return re.sub("\s\s+" , " ",text).strip(" ")
##~4
def remove_User(text):
    return re.sub(r'@([^\s:]+)', "", text)

##5
def removeDigits(text):
        return ' '.join([w for w in word_tokenize(str(text)) if not re.search(r'\d', w)])

#6
def correct_spellings(text):
    spell = SpellChecker()
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

##7
def clearPunct(text):
        text=text.replace("#","").replace('¿','')
        # a = re.sub(r'[^\w\s]',' ',text)
        text=text.translate(str.maketrans('', '', string.punctuation))
        return text.replace("_","")
    

##8
def lemmatise(text):
    doc=nlp(text.lower())
    lemm_list=[token.lemma_ for token in doc]
    return " ".join(lemm_list)

In [4]:
def preprocess_text_tweet(self, text):
        i = self.clearPunct(self.removeDigits(self.clean_text(self.removeEmoji(self.remove_NE(text))).lower()))
        return self.clearSpaces(self.removeStop(self.lemmatize(i)))

In [5]:
def pre_tweet(tweet):
    tweet=tweet.lower()
    tweet=remove_User(tweet)
    tweet=remove_Urls(tweet)
    tweet=removeDigits(tweet)
    tweet=clearPunct(tweet)
    tweet=remove_stopwords(tweet)
    tweet=lemmatise(tweet)
    tweet=clear_spaces(tweet)
    return tweet

In [6]:
list(pd.read_csv('train.csv')['text'])[1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
train=pd.read_csv('train.csv')

In [8]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [131]:
pre_tweet(list(pd.read_csv('train.csv')['text'])[1])

'forest fire near la ronge sask canada'

In [12]:
train['prep_text'] = train['text'].progress_apply(lambda x: pre_tweet(x))

100%|██████████| 7613/7613 [02:06<00:00, 60.41it/s] 


## BOW+models

In [10]:
pd.read_csv('prep_train')

FileNotFoundError: [Errno 2] File b'prep_train' does not exist: b'prep_train'

In [49]:
pre_tweet('Err:509')

''

In [53]:
train.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [14]:
X = train['prep_text']
y = train['target']

In [15]:
count_vectorizer = feature_extraction.text.CountVectorizer()
X_train_=count_vectorizer.fit_transform(X)


In [186]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=122)

In [187]:
#to generate bag of words
count_vectorizer = feature_extraction.text.CountVectorizer()

In [188]:
train_vectors = count_vectorizer.fit_transform(X_train)
test_vectors = count_vectorizer.transform(X_test)

In [115]:
print(train_vectors.shape)
print(test_vectors.shape)

(5329, 9467)
(2284, 9467)


In [191]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [196]:
models=[MultinomialNB(fit_prior=False),
SVC(gamma='scale'),
RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0), #
LogisticRegression(random_state=0), 
GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0),
KNeighborsClassifier(3),
SVC(kernel="linear", C=0.025,verbose=True),
SVC(gamma=2, C=1),
#GaussianProcessClassifier(1.0 * RBF(1.0)),
DecisionTreeClassifier(max_depth=5),
RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
MLPClassifier(alpha=1, max_iter=200,tol=0.001),
AdaBoostClassifier()]
for i in models:
    model=i.fit(train_vectors, y_train)
    y_pred = model.predict(test_vectors)
    print(str(i))
    print(f1_score(y_test, y_pred, average='micro'))
    print(confusion_matrix(y_test, y_pred))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)
0.7819614711033275
[[1053  225]
 [ 273  733]]
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.7841506129597198
[[1168  110]
 [ 383  623]]
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
0.5626094570928196
[[1278    0]
 [ 999    7]]


C:\Users\asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.7837127845884413
[[1096  182]
 [ 312  694]]
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=1.0, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
    

### TF_IDF+model

In [69]:
tfidf_transformer = TfidfTransformer()
train_tfidf = tfidf_transformer.fit_transform(train_vectors)
test_tfidf = tfidf_transformer.fit_transform(test_vectors)

In [70]:
for i in models:
    model=i.fit(train_tfidf, y_train)
    y_pred = model.predict(test_tfidf)
    print(str(i))
    print(f1_score(y_test, y_pred, average='micro'))
    print(f1_score(y_test, y_pred, average='micro'))

    print(confusion_matrix(y_test, y_pred))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.7885288966725044
[[1151  127]
 [ 356  650]]
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.780647985989492
[[1151  127]
 [ 374  632]]
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
0.5678633975481612
[[1278    0]
 [ 987   19]]
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_

C:\Users\asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.7828371278458844
[[1129  149]
 [ 347  659]]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
0.7158493870402802
[[953 325]
 [324 682]]
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impu

# In order to try the test ... but this is after the preprocessing

In [72]:
test_df = pd.read_csv('test.csv')
gt_df = pd.read_csv("socialmedia-disaster-tweets-DFE.csv")


gt_df = gt_df[['choose_one', 'text']]
gt_df['target'] = (gt_df['choose_one']=='Relevant').astype(int)
gt_df['id'] = gt_df.index
gt_df

merged_df = pd.merge(test_df, gt_df, on='id')
merged_df

subm_df = merged_df[['id', 'target']]
subm_df

subm_df.to_csv('submission.csv', index=False)

In [89]:
real=list(subm_df['target'])

In [77]:
X = train['prep_text']
y = train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=122)


In [95]:
train_vectors = count_vectorizer.fit_transform(X)
train_tfidf = tfidf_transformer.fit_transform(train_vectors)

In [92]:
s=pd.read_csv('test.csv')
s['target']=real
s['prep_text']=s['text'].progress_apply(lambda x: pre_tweet(x))

100%|██████████| 3263/3263 [00:32<00:00, 101.87it/s]


In [94]:
test_vectors = count_vectorizer.transform(s['prep_text'])
test_tfidf = tfidf_transformer.fit_transform(test_vectors)

In [96]:
c=MultinomialNB().fit(train_tfidf,y)

In [119]:
y_pred = c.predict(test_tfidf)

In [120]:
f1_score(s['target'], y_pred, average='micro')

0.7925222188170395

In [121]:
f1_score(y_pred,s['target'], average='micro')

0.7925222188170395

### ....
### let s start a grid search

In [100]:
MultinomialNB()

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [111]:
list(grid)

[{'a': 1, 'b': 2},
 {'a': 1, 'b': 2},
 {'a': 2, 'b': 2},
 {'a': 2, 'b': 2},
 {'a': 3, 'b': 2},
 {'a': 3, 'b': 2},
 {'a': 3, 'b': 2},
 {'a': 3, 'b': 2}]

In [118]:
MultinomialNB()

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
accuracy_score(y_true, y_pred)

In [180]:
hyper_comb={'alpha':[0.005,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.5],
            'fit_prior':[True,False]} ##combination of hyperparameters

def MultinomialNB_grid_search(hyper_comb,X_train,y_train):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in grid:
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        alpha = hypers['alpha']
        fit_prior = hypers['fit_prior']

        for train_index, test_index in cv.split(X_train): # cross validation
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen parameters
            model=MultinomialNB(alpha=alpha,fit_prior=fit_prior).fit(X_train_cv,y_train_cv)
            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)
            
        all_res_cv.append({'alpha':alpha,
                           'fit_prior':fit_prior,
                           'val_f1':np.mean(val_f1_list), 
                           'train_f1':np.mean(train_f1_list),
                           'val_recall':np.mean(val_recall_list), 
                           'train_recall':np.mean(train_recall_list),
                           'val_acc':np.mean(val_accuracy_list), 
                           'train_acc':np.mean(train_accuracy_list),
                           'val_pre':np.mean(val_pre_list),
                          'train_pre':np.mean(train_pre_list)})
        print(({'alpha':alpha,'fit_prior':fit_prior,'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))

        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv('multinomial_grid_search_cv.csv', index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')


In [181]:
MultinomialNB_grid_search(hyper_comb,X_train_,y)

{'alpha': 0.005, 'fit_prior': True, 'val_f1': 0.7726249198664714, 'train_f1': 0.9443386255367754}
{'alpha': 0.005, 'fit_prior': False, 'val_f1': 0.7642182979681675, 'train_f1': 0.9424338905904133}
{'alpha': 0.25, 'fit_prior': True, 'val_f1': 0.7902272038985231, 'train_f1': 0.9220412199322858}
{'alpha': 0.25, 'fit_prior': False, 'val_f1': 0.7818209271244336, 'train_f1': 0.919742482582852}
{'alpha': 0.5, 'fit_prior': True, 'val_f1': 0.7952183902888948, 'train_f1': 0.9118940890743268}
{'alpha': 0.5, 'fit_prior': False, 'val_f1': 0.7877309204549083, 'train_f1': 0.9088729582719018}
{'alpha': 0.75, 'fit_prior': True, 'val_f1': 0.7981081153370613, 'train_f1': 0.9039800141208097}
{'alpha': 0.75, 'fit_prior': False, 'val_f1': 0.7919343608256406, 'train_f1': 0.9017141336689114}
{'alpha': 1, 'fit_prior': True, 'val_f1': 0.7988962927619687, 'train_f1': 0.8971495751760585}
{'alpha': 1, 'fit_prior': False, 'val_f1': 0.7942983754140412, 'train_f1': 0.8954748169457266}
{'alpha': 1.25, 'fit_prior': Tru

In [165]:
set(train['target'])

{0, 1}

In [198]:
SVC(gamma='scale'),

(SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=None, shrinking=True,
     tol=0.001, verbose=False),)

In [6]:
'''
hyper_comb={'C':[0.005,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.5],
            'coef0':[0.005,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.5],
              'kernel':['linear','poly','rbf','sigmoid'],
           'degree':[1,2,3,4,5],
           'gamma':['scale','auto',0.005,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3],
           'shrinking':[True,False],
           'decision_function_shape':['ovo','ovr']} ##combination of hyperparameters
'''

hyper_comb={'C':[0.005],
            'coef0':[0.005,0.25],
              'kernel':['linear','poly','rbf','sigmoid'],
           'degree':[1,2,3],
           'gamma':['scale','auto'],
           'shrinking':[True,False],
           'decision_function_shape':['ovo','ovr']} ##combination of hyperparameters
def SVC_grid_search(hyper_comb,X_train,y_train):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in grid:
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        C = hypers['C']
        coef0 = hypers['coef0']
        kernel=hypers['kernel']
        degree=hypers['degree']
        gamma=hypers['gamma']
        shrinking=hypers['shrinking']
        decision_function_shape=hypers['decision_function_shape']

        for train_index, test_index in cv.split(X_train): # cross validation
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen parameters
            model=SVC(C=C,coef0=coef0,kernel=kernel,degree=degree,gamma=gamma,shrinking=shrinking,decision_function_shape=decision_function_shape,random_state=1848773).fit(X_train_cv,y_train_cv)
            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)
            
        all_res_cv.append({'C':C,
                           'coef0' :coef0,
                            'kernel':kernel,
                            'degree':degree,
                            'gamma':gamma,
                        'shrinking':shrinking,
                            'decision_function_shape':decision_function_shape,

                           'val_f1':np.mean(val_f1_list), 
                           'train_f1':np.mean(train_f1_list),
                           'val_recall':np.mean(val_recall_list), 
                           'train_recall':np.mean(train_recall_list),
                           'val_acc':np.mean(val_accuracy_list), 
                           'train_acc':np.mean(train_accuracy_list),
                           'val_pre':np.mean(val_pre_list),
                          'train_pre':np.mean(train_pre_list)})
        print(({'C':C,
                           'coef0' :coef0,
                            'kernel':kernel,
                            'degree':degree,
                            'gamma':gamma,
                        'shrinking':shrinking,
                            'decision_function_shape':decision_function_shape,'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))

        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv('SVC_grid_search_cv.csv', index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')


#SVC_grid_search(hyper_comb,X_train_,y)

In [26]:
hyper_comb={'C':[0.005],
            'dual':[True,False],
              'fit_intercept':[True,False],
          'intercept_scaling':[0.5,1,1.5],
            'l1_ratio':[0.5],
           'max_iter':[500],
           'multi_class':['auto','ovr','multinomial'],
            'penalty':['l1','l2','elasticnet','none'],
           'warm_start':[True,False],
           'random_state':[1848773],
           'solver':['newton-cg','lbfgs','liblinear','sag','saga']}

def log_grid_search(hyper_comb,X_train,y_train):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in grid:
        print(hypers)
        C = hypers['C']
        dual = hypers['dual']
        fit_intercept=hypers['fit_intercept']
        intercept_scaling=hypers['intercept_scaling']
        l1_ratio=hypers['l1_ratio']
        max_iter=hypers['max_iter']
        multi_class=hypers['multi_class']
        penalty=hypers['penalty']
        warm_start=hypers['warm_start']
        random_state=hypers['random_state']
        solver=hypers['solver']
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        

        for train_index, test_index in cv.split(X_train): # cross validation
            try:
                #take 80% and 20%
                X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
                #fit the model with the chosen parameters
                model=LogisticRegression(C=C, class_weight=None, dual=dual, fit_intercept=fit_intercept,
                       intercept_scaling=intercept_scaling, l1_ratio=l1_ratio, max_iter=max_iter,
                       multi_class=multi_class, n_jobs=None, penalty=penalty,
                       random_state=random_state, solver=solver, tol=0.0001,
                       warm_start=warm_start).fit(X_train_cv,y_train_cv)
                # make prediction and compute F1
                pred_train = model.predict(X_train_cv)
                pred_val = model.predict(X_val)
                #F1 SCORE
                f1_train = f1_score(y_train_cv, pred_train, average='micro')
                train_f1_list.append(f1_train)
                f1_val = f1_score(y_val, pred_val, average='micro')
                val_f1_list.append(f1_val)
                #RECALL VALUES
                recall_train=recall_score(y_train_cv,pred_train, average='micro')
                train_recall_list.append(recall_train)
                recall_val=recall_score(y_val,pred_val, average='micro')
                val_recall_list.append(recall_val)
                #ACCURACY
                accuracy_train=accuracy_score(y_train_cv,pred_train)
                train_accuracy_list.append(accuracy_train)
                accuracy_val=accuracy_score(y_val,pred_val)
                val_accuracy_list.append(accuracy_val)
                #PRECISION
                pre_train=precision_score(y_train_cv,pred_train, average='micro')
                train_pre_list.append(pre_train)
                pre_val=precision_score(y_val,pred_val, average='micro')
                val_pre_list.append(pre_val)



                all_res_cv.append({'C':C,
                                   'dual' :dual,
                                    'fit_intercept':fit_intercept,
                                    'intercept_scaling':intercept_scaling,
                                    'l1_ratio':l1_ratio,
                                'max_iter':max_iter,
                                    'multi_class':decision_function_shape,
                                   'penalty':penalty,
                                   'warm_start':warm_start,
                                   'random_state':random_state,
                                   'solver':solver,
                                   'val_f1':np.mean(val_f1_list), 
                                   'train_f1':np.mean(train_f1_list),
                                   'val_recall':np.mean(val_recall_list), 
                                   'train_recall':np.mean(train_recall_list),
                                   'val_acc':np.mean(val_accuracy_list), 
                                   'train_acc':np.mean(train_accuracy_list),
                                   'val_pre':np.mean(val_pre_list),
                                  'train_pre':np.mean(train_pre_list)})
                print(({'C':C,
                                   'dual' :dual,
                                    'fit_intercept':fit_intercept,
                                    'intercept_scaling':intercept_scaling,
                                    'l1_ratio':l1_ratio,
                                'max_iter':max_iter,
                                    'multi_class':decision_function_shape,
                                   'penalty':penalty,
                                   'warm_start':warm_start,
                                   'random_state':random_state,
                                   'solver':solver,'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))
            except:
                print('excep')
                pass

        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv('log_grid_search_cv.csv', index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')

##combination of hyperparameters

In [ ]:
MLPClassifier(activation='relu', alpha=1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.001,
              validation_fraction=0.1, verbose=False, warm_start=False)


In [ ]:
hyper_comb={'activation':['identity','logistic','tanh','relu'],
'alpha':[0.001,0.005,0.01],
'batch_size':['auto',300,400],
'beta_1':[0.4,0.5,0.9],
'beta_2':[0,0.5],
'early_stopping':[True,False],
'learning_rate_init':[0.001,0.004,0.008],
'max_iter':[100,200,300],
'momentum':[0,0.1,0.3,0.5,0.7],
'n_iter_no_change':[10,20],
'nesterovs_momentum':[True,False],
'power_t':[0.5,0.3]
'random_state':[1848773],
'shuffle':[True,False],
'solver':['lbfgs','sgd','adam'],
'tol':[1e-3],
'validation_fraction':[0.1,0.3,0.5]}
    



In [32]:
hyper_comb={'activation':['identity','logistic','tanh','relu'],
'alpha':[0.001,0.005,0.01],
'batch_size':['auto',300,400],
'beta_1':[0.4,0.5,0.9],
'beta_2':[0,0.5],
'early_stopping':[True,False],
'learning_rate_init':[0.001,0.004,0.008],
'max_iter':[100,200,300],
'momentum':[0,0.1,0.3,0.5,0.7],
'n_iter_no_change':[10,20],
'nesterovs_momentum':[True,False],
'power_t':[0.5,0.3],
'random_state':[1848773],
'shuffle':[True,False],
'solver':['lbfgs','sgd','adam'],
'tol':[1e-3],
'validation_fraction':[0.1,0.3,0.5],
'hidden_layer_sizes':[(100,)],
'learning_rate':[0.001],
'warm_start':[True]}

def mlp_grid_search(hyper_comb,X_train,y_train):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in grid:
        activation=hypers['activation']
        alpha=hypers['alpha']
        batch_size=hypers['batch_size']
        beta_1=hypers['beta_1']
        beta_2=hypers['beta_2']
        early_stopping=hypers['early_stopping']
        learning_rate_init=hypers['learning_rate_init']
        learning_rate=hypers['learning_rate']
        max_iter=hypers['max_iter']
        momentum=hypers['momentum']
        warm_start=hypers['warm_start']
        n_iter_no_change=hypers['n_iter_no_change']
        nesterovs_momentum=hypers['nesterovs_momentum']
        power_t=hypers['power_t']
        random_state=hypers['random_state']
        shuffle=hypers['shuffle']
        solver=hypers['solver']
        tol=hypers['tol']
        validation_fraction=hypers['validation_fraction']
        hidden_layer_sizes=hypers['hidden_layer_sizes']
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        

        for train_index, test_index in cv.split(X_train): # cross validation
            
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen parameters
            model=MLPClassifier(activation=activation, alpha=alpha, batch_size=batch_size, beta_1=beta_1,
              beta_2=beta_2, early_stopping=early_stopping, epsilon=1e-08,
              hidden_layer_sizes=hidden_layer_sizes, learning_rate=learning_rate,
              learning_rate_init=learning_rate_init, max_iter=max_iter, momentum=momentum,
              n_iter_no_change=n_iter_no_changes, nesterovs_momentum=nesterovs_momentum, power_t=power_t,
              random_state=random_state, shuffle=shuffle, solver=solver, tol=tol,
              validation_fraction=validation_fraction, verbose=False, warm_start=warm_start).fit(X_train_cv,y_train_cv)

            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)



            all_res_cv.append({'activation':activation,
                                'alpha':alpha,
                                'batch_size':batch_size,
                                'beta_1':beta_1,
                                'beta_2':beta_2,
                                'early_stopping':early_stopping,
                                'learning_rate_init':learning_rate_init,
                                'max_iter':max_iter,
                                'momentum':momentum,
                                'n_iter_no_change':n_iter_no_change,
                                'nesterovs_momentum':nesterovs_momentum,
                                'power_t':power_t,
                                'random_state':random_state,
                                'shuffle':shuffle,
                                'solver':solver,
                                'tol':tol,
                                'validation_fraction':validation_fraction,
                                'hidden_layer_sizes':hidden_layer_sizes,
                                'learning_rate':learning_rate,
                                'warm_start':warm_start,
                               'val_f1':np.mean(val_f1_list), 
                               'train_f1':np.mean(train_f1_list),
                               'val_recall':np.mean(val_recall_list), 
                               'train_recall':np.mean(train_recall_list),
                               'val_acc':np.mean(val_accuracy_list), 
                               'train_acc':np.mean(train_accuracy_list),
                               'val_pre':np.mean(val_pre_list),
                              'train_pre':np.mean(train_pre_list)})
            print(({'activation':activation,
                                'alpha':alpha,
                                'batch_size':batch_size,
                                'beta_1':beta_1,
                                'beta_2':beta_2,
                                'early_stopping':early_stopping,
                                'learning_rate_init':learning_rate_init,
                                'max_iter':max_iter,
                                'momentum':momentum,
                                'n_iter_no_change':n_iter_no_change,
                                'nesterovs_momentum':nesterovs_momentum,
                                'power_t':power_t,
                                'random_state':random_state,
                                'shuffle':shuffle,
                                'solver':solver,
                                'tol':tol,
                                'validation_fraction':validation_fraction,
                                'hidden_layer_sizes':hidden_layer_sizes,
                                'learning_rate':learning_rate,
                                'warm_start':warm_start,
                'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))


        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv('mlp_grid_search_cv.csv', index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')

##combination of hyperparameters

In [ ]:
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,n_estimators=50, random_state=None)

In [33]:
hyper_comb={ 'learning_rate':[0.5,1.0,2.0],
            'n_estimators':[50,60,70]}
#THE HYPERPARAMETER BASE_ESTIMATOR CAN BE CHANGED BUT FOR SIMPLICITY
def ada_grid_search(hyper_comb,X_train,y_train):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in grid:
        learning_rate=hypers['learning_rate']
        n_estimators=hypers['n_estimators']
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        

        for train_index, test_index in cv.split(X_train): # cross validation
            
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen parameters
            model=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=learning_rate,n_estimators=n_estimators, random_state=1848773).fit(X_train_cv,y_train_cv)

            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)



            all_res_cv.append({'learning_rate':learning_rate,
                                'n_estimators':n_estimators,
                               'val_f1':np.mean(val_f1_list), 
                               'train_f1':np.mean(train_f1_list),
                               'val_recall':np.mean(val_recall_list), 
                               'train_recall':np.mean(train_recall_list),
                               'val_acc':np.mean(val_accuracy_list), 
                               'train_acc':np.mean(train_accuracy_list),
                               'val_pre':np.mean(val_pre_list),
                              'train_pre':np.mean(train_pre_list)})
            print(({'learning_rate':learning_rate,
                                'n_estimators':n_estimators,
                'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))


        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv('ada_grid_search_cv.csv', index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')

##combination of hyperparameters

In [ ]:
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')


In [37]:
hyper_comb={ 'algorithm':['auto','ball_tree','kd_tree','brute'],
             'leaf_size':[30],
             'metric':['minkowski'],
             'n_neighbors':[2,3,4],
            'p':[1,2,3],
           'weighths':['uniform','distance']}

dict_keys(['algorithm', 'leaf_size', 'metric', 'n_neighbors', 'p', 'weighths'])

In [40]:
hyper_comb={ 'algorithm':['auto','ball_tree','kd_tree','brute'],
             'leaf_size':[30],
             'metric':['minkowski'],
             'n_neighbors':[2,3,4],
            'p':[1,2,3],
           'weighths':['uniform','distance']}

#THE HYPERPARAMETER BASE_ESTIMATOR CAN BE CHANGED BUT FOR SIMPLICITY
def KNC_grid_search(hyper_comb,X_train,y_train):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in grid:
        algorithm=hypers['algorithm']
        leaf_size=hypers['leaf_size']
        metric=hypers['metric']
        n_neighbors=hypers['n_neighbors']
        p=hypers['p']
        weights=hypers['weights']
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        

        for train_index, test_index in cv.split(X_train): # cross validation
            
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen parameters
            model=KNeighborsClassifier(algorithm=algorithm, leaf_size=leaf_size, metric=metric, n_jobs=None, n_neighbors=n_neighbors, p=p,
                     weights=weights).fit(X_train_cv,y_train_cv)

            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)



            all_res_cv.append({ 'algorithm':algorithm,
                                 'leaf_size':leaf_size,
                                 'metric':metric,
                                 'n_neighbors':n_neighbors,
                                    'p':p,
                                   'weighths':weights,
                               'val_f1':np.mean(val_f1_list), 
                               'train_f1':np.mean(train_f1_list),
                               'val_recall':np.mean(val_recall_list), 
                               'train_recall':np.mean(train_recall_list),
                               'val_acc':np.mean(val_accuracy_list), 
                               'train_acc':np.mean(train_accuracy_list),
                               'val_pre':np.mean(val_pre_list),
                              'train_pre':np.mean(train_pre_list)})
            print(({'algorithm':algorithm,
                                 'leaf_size':leaf_size,
                                 'metric':metric,
                                 'n_neighbors':n_neighbors,
                                    'p':p,
                                   'weighths':weights,
                'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))


        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv('knc_grid_search_cv.csv', index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')

##combination of hyperparameters

In [ ]:
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=1.0, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [45]:
hyper_comb={'criterion':['friedman_mse'],
            'init':[None,'zero'],
           'learning_rate':[0.1,0.4],
            'loss':['deviance','exponential'],
            'max_depth':[1,2],
            'max_features':['auto','sqrt','log2',None],
            'max_leaf_nodes':[None],
            'min_impurity_decrease':[0],
            'min_impurity_split':[None],
            'min_samples_leaf':[1],
            'min_samples_split':[2],
            'min_weight_fraction_leaf':[0.0],
            'n_estimators':[100],
            'n_iter_no_change':[None],
            'presort':['deprecated'],
            'subsample':[1,2,3],
            'tol':[0.001],
            'validation_fraction':[0.1],
            'warm_start':[True,False]}

#THE HYPERPARAMETER BASE_ESTIMATOR CAN BE CHANGED BUT FOR SIMPLICITY
def GBC_grid_search(hyper_comb,X_train,y_train):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in grid:
        criterion=hypers['criterion']
        init=hypers['init']
        learning_rate=hypers['learning_rate']
        loss=hypers['loss']
        max_depth=hypers['max_depth']
        max_features=hypers['max_features']
        max_leaf_nodes=hypers['max_leaf_nodes']
        min_impurity=hypers['min_impurity_decrease']
        min_impurity_split=hypers['min_impurity_split']
        min_samples_leaf=hypers['min_samples_leaf']
        min_samples_split=hypers['min_samples_split']
        min_weight_fraction_leaf=hypers['min_weight_fraction_leaf']
        n_estimators= hypers['n_estimators']
        n_iter_no_change=hypers['n_iter_no_change']
        presort=hypers['presort']
        subsample=hypers['subsample']
        tol=hypers['tol']
        validation_fraction=hypers['validation_fraction']
        warm_start=hypers['warm_start']
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        

        for train_index, test_index in cv.split(X_train): # cross validation
            
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen param1848773eters
            model=GradientBoostingClassifier(criterion=criterion, init=init,
                           learning_rate=learning_rate, loss=loss, max_depth=max_depth,
                           max_features=max_features, max_leaf_nodes=max_leaf_nodes,
                           min_impurity_decrease=min_impurity_decrease, min_impurity_split=min_impurity_split,
                           min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,
                           min_weight_fraction_leaf=min_weight_fraction_leaf, n_estimators=n_estimators,
                           n_iter_no_change=n_iter_no_change, presort=presort,
                           random_state=, subsample=subsample, tol=tol,
                           validation_fraction=validation_fraction, verbose=0,
                           warm_start=warn_start).fit(X_train_cv,y_train_cv)

            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)



        all_res_cv.append({ 'criterion':criterion,
        'init':init,
        'learning_rate':learning_rate,
        'loss':loss,
        'max_depth':max_depth,
        'max_features':max_features,
        'max_leaf_nodes':max_leaf_nodes,
        'min_impurity':min_impurity_decrease,
        'min_impurity_split':min_impurity_split,
        'min_samples_leaf':min_samples_leaf,
        'min_samples_split':min_samples_split,
        'min_weight_fraction_leaf':min_weight_fraction_leaf,
        'n_estimators':n_estimators,
        'n_iter_no_change':n_iter_no_change,
        'presort':presort,
        'subsample':subsample,
        'tol':tol,
        'validation_fraction':validation_fraction,
        'warm_start':warm_start,
        'val_f1':np.mean(val_f1_list), 
        'train_f1':np.mean(train_f1_list),
        'val_recall':np.mean(val_recall_list), 
        'train_recall':np.mean(train_recall_list),
        'val_acc':np.mean(val_accuracy_list), 
        'train_acc':np.mean(train_accuracy_list),
        'val_pre':np.mean(val_pre_list),
        'train_pre':np.mean(train_pre_list)})
        print(({'criterion':criterion,
        'init':init,
        'learning_rate':learning_rate,
        'loss':loss,
        'max_depth':max_depth,
        'max_features':max_features,
        'max_leaf_nodes':max_leaf_nodes,
        'min_impurity':min_impurity_decrease,
        'min_impurity_split':min_impurity_split,
        'min_samples_leaf':min_samples_leaf,
        'min_samples_split':min_samples_split,
        'min_weight_fraction_leaf':min_weight_fraction_leaf,
        'n_estimators':n_estimators,
        'n_iter_no_change':n_iter_no_change,
        'presort':presort,
        'subsample':subsample,
        'tol':tol,
        'validation_fraction':validation_fraction,
        'warm_start':warm_start,
            'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))


        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv('gbc_grid_search_cv.csv', index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')

##combination of hyperparameters

In [ ]:
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


In [48]:
hyper_comb={'class_weight':[None,'balanced'],
            'criterion':['gini'],
            'max_depth':[None],
            'max_features':[None],
            'max_leaf_nodes':[None],
            'min_impurity_decrease':[0],
            'min_impurity_split':[None],
            'min_samples_leaf':[1],
            'min_samples_split':[2],
            'min_weight_fraction_leaf':[0.0],
            'presort':[False],
            'splitter':['best']}
    
def DT_grid_search(hyper_comb,X_train,y_train):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in grid:
        class_weight=hypers['class_weight'],
        criterion=hypers['criterion']
        max_depth=hypers['max_depth']
        max_features=hypers['max_features']
        max_leaf_nodes=hypers['max_leaf_nodes']
        min_impurity_decrease=hypers['min_impurity_decrease']
        min_impurity_split=hypers['min_impurity_split']
        min_samples_leaf=hypers['min_samples_leaf']
        min_samples_split=hypers['min_samples_split']
        min_weight_fraction_leaf=hypers['min_weigth_fraction_leaf']
        presort=hypers['presort']
        splitter=hypers['splitter']
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        

        for train_index, test_index in cv.split(X_train): # cross validation
            
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen param1848773eters
            model=DecisionTreeClassifier(class_weight=class_weight, criterion=criterion, max_depth=max_depth,
                       max_features=max_features, max_leaf_nodes=max_leaf_nodes,
                       min_impurity_decrease=min_impurity_decrease, min_impurity_split=min_impurity_split,
                       min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,
                       min_weight_fraction_leaf=min_weight_fraction_leaf, presort=presort,
                       random_state=1848773, splitter=splitter).fit(X_train_cv,y_train_cv)

            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)



        all_res_cv.append({ 'class_weight':class_weight,
            'criterion':criterion,
            'max_depth':max_depth,
            'max_features':max_features,
            'max_leaf_nodes':max_leaf_nodes,
            'min_impurity_decrease':min_impurity_decrease,
            'min_impurity_split':min_impurity_split,
            'min_samples_leaf':min_samples_leaf,
            'min_samples_split':min_samples_split,
            'min_weight_fraction_leaf':min_weight_fraction_leaf,
            'presort':presort,
            'splitter':splitter,
        'val_f1':np.mean(val_f1_list), 
        'train_f1':np.mean(train_f1_list),
        'val_recall':np.mean(val_recall_list), 
        'train_recall':np.mean(train_recall_list),
        'val_acc':np.mean(val_accuracy_list), 
        'train_acc':np.mean(train_accuracy_list),
        'val_pre':np.mean(val_pre_list),
        'train_pre':np.mean(train_pre_list)})
        print(({'class_weight':class_weight,
            'criterion':criterion,
            'max_depth':max_depth,
            'max_features':max_features,
            'max_leaf_nodes':max_leaf_nodes,
            'min_impurity_decrease':min_impurity_decrease,
            'min_impurity_split':min_impurity_split,
            'min_samples_leaf':min_samples_leaf,
            'min_samples_split':min_samples_split,
            'min_weight_fraction_leaf':min_weight_fraction_leaf,
            'presort':presort,
            'splitter':splitter,
            'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))


        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv('DT_grid_search_cv.csv', index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')

##combination of hyperparameters